<a href="https://colab.research.google.com/github/danjethh/steg_analysis/blob/main/steg_analysis_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run the script. It will:
Load and preprocess the dataset.
Train the Random Forest Classifier.
Evaluate the model on the test set.
Prompt you to enter the path to an image for testing.
 Enter the path to the image you want to test when prompted. Ensure the image is 512x512 pixels.

Workflow Summary

**Step 1:**
1. Load the clean dataset - 10,000 features dataset
2. Load the stego datasets - 10,000 features dataset
3. Combine them into a single DataFrame.
4. Add labels to distinguish between clean and stego images.

**Step 2:**
Preprocess the combined 20,000 dataset
1. Removing rows containing NaN values (invalid computations or uniform features).
2. Normalizing features using StandardScaler (to ensure zero mean and unit variance).
3. Reducing dimensionality using PCA (retains 10 most important components).
4. Train classifier
5. Extract CF features from image provided for prediction
6. Predict image: Clean ro Steg

In [2]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 80.8 MB/s eta 0:00:00


In [3]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

## Step 2: Load the Dataset Function

### Input
- No external input from the user.
- Internally fetches two datasets from URLs:
  - Clean image features (`steg_features.csv`)
  - Stego image features (`steg_lsb_features.csv`)

### Output
- A **combined dataset** (DataFrame) containing 20,000 rows (10,000 clean + 10,000 stego) with:
  - 41 feature columns
  - 1 label column (`0` for clean, `1` for stego)

### Brief Explanation
This function loads pre-extracted feature vectors for clean and stego images. Each image is represented by **41 statistical and transformation-based features**. After loading:

1. Clean images are labeled with `0`, and stego images with `1`.
2. The datasets are then **concatenated into one unified table**.
3. First 4 rows of each set and first 8 rows of the combined dataset are printed.
4. The final dataset (20,000 rows × 42 columns) is returned for further preprocessing and machine learning.

This helps students see what raw feature data looks like **before any processing or training begins**.


In [4]:
# Step 2: Load the Dataset Function
def load_data():
    """
    Loads the clean (cover) and stego image feature datasets, labels them,
    combines them, and displays preview outputs for students to understand.
    Returns the full combined dataset with labels.
    """

    # URLs for clean and stego datasets (CSV with 41 features each)
    url_clean = "https://raw.githubusercontent.com/Sourish1997/steganalysis/master/Datasets/steg_features.csv"
    url_stego = "https://raw.githubusercontent.com/Sourish1997/steganalysis/master/Datasets/steg_lsb_features.csv"

    # Load clean (cover) images feature dataset
    print("Loading clean (cover) dataset...")
    data_clean = pd.read_csv(url_clean, header=None)
    data_clean['label'] = 0  # Label '0' for clean images

    # Display first 4 rows of the 10,000 rows
    print("\nFirst 4 rows from Clean (Cover) Dataset:")
    print(data_clean.head(4))

    # Load stego images feature dataset
    print("\nLoading stego dataset...")
    data_stego = pd.read_csv(url_stego, header=None)
    data_stego['label'] = 1  # Label '1' for stego images

    # Display first 4 rows of the 10,000 rows from stego dataset
    print("\nFirst 4 rows from Stego Dataset:")
    print(data_stego.head(4))

    # Combine both datasets
    print("\nCombining clean and stego datasets into a single DataFrame...")
    data_combined = pd.concat([data_clean, data_stego], axis=0, ignore_index=True)

    # Display first 8 rows of the combined dataset with labels
    print("\nFirst 8 rows of the Combined Dataset (including labels):")
    print(data_combined.head(8))

    # Display the shape of the combined dataset
    print(f"\nCombined Dataset Shape: {data_combined.shape}")

    return data_combined  # Return full dataset (100%) without sampling

# Run the function
full_dataset = load_data()

Loading clean (cover) dataset...

First 4 rows from Clean (Cover) Dataset:
          0         1         2         3         4         5         6  \
0 -0.317327  0.827515  0.760605  0.740966  0.721418  0.910647  0.861356   
1       NaN       NaN       NaN       NaN       NaN       NaN       NaN   
2 -0.503111  0.862970  0.802899  0.775813  0.751000  0.927452  0.889261   
3 -0.182988  0.887022  0.835196  0.813357  0.789932  0.911072  0.861291   

          7         8         9  ...        32        33        34        35  \
0  0.835196  0.815543  0.818339  ... -0.004257 -0.000239 -0.266943 -0.106837   
1       NaN       NaN       NaN  ... -0.064528  0.015347  0.005049 -0.145678   
2  0.866067  0.848226  0.855546  ...  0.003529  0.009316 -0.248362 -0.107545   
3  0.824739  0.795830  0.856713  ... -0.024424  0.004261 -0.137704 -0.088573   

         36        37        38        39        40  label  
0 -0.059703 -0.015162 -0.006729 -0.004329  0.001190      0  
1 -0.189235  0.075486  0.0

## Step 3: Preprocess the Dataset

### Input
- `data`: A DataFrame combining both clean and stego images with 41 feature columns and a `label` column.

### Output
- `X`: Processed feature matrix (after cleaning, normalization, and PCA).
- `y`: Corresponding labels (`0` = clean, `1` = stego).
- `scaler`: Fitted `StandardScaler` object (used to normalize future input data).
- `pca`: Fitted PCA object (used to reduce future data to 10 key components).

### Brief Explanation
This function prepares the dataset for machine learning by performing the following steps:

1. **Remove invalid rows**: Any row with a missing or undefined value (NaN) is dropped.
2. **Normalize features**: Standardizes each feature so it has a mean of 0 and standard deviation of 1. This ensures equal treatment of all features.
3. **Dimensionality Reduction (PCA)**: Compresses the 41 feature dimensions down to the top 10 principal components. These components capture the most important patterns in the data while reducing redundancy.

This helps simplify the dataset, improve learning efficiency, and avoid overfitting.



In [5]:
# Step 2: Preprocess the Dataset
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np

# Function to preprocess the data
def preprocess_data(data):
    """
    Preprocess the dataset by:
    1. Removing rows containing NaN values (invalid computations or uniform features).
    2. Normalizing features using StandardScaler (to ensure zero mean and unit variance).
    3. Reducing dimensionality using PCA (retains 10 most important components).

    Returns:
    - X: Preprocessed features
    - y: Labels (0 = clean, 1 = stego)
    - scaler: Fitted StandardScaler object
    - pca: Fitted PCA object
    """

    # Step 1: Split dataset into features and labels
    X = data.drop(columns=['label']).values  # Drop label column for features
    y = data['label'].values  # Extract label column

    # Step 2: Remove any row that contains NaN (can happen due to uniform images or divide by zero)
    print("\nRemoving rows with NaN values...")
    nan_mask = ~np.isnan(X).any(axis=1)  # Mask where rows do not contain NaNs
    X = X[nan_mask]
    y = y[nan_mask]
    print(f"Dataset shape after removing NaNs: {X.shape}")
    print("First 5 rows of X (after NaN removal):")
    print(X[:5])  # Show first 5 rows

    # Step 3: Normalize the features to have mean=0 and std=1
    print("\nNormalizing features with StandardScaler...")
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    print("First 5 rows after normalization:")
    print(X[:5])

    # Step 4: Apply PCA to reduce to top 10 most significant components
    print("\n Applying PCA to reduce dimensionality to 10 components...")
    pca = PCA(n_components=10)
    X = pca.fit_transform(X)
    print("Explained Variance Ratio of PCA:")
    print(pca.explained_variance_ratio_)
    print("First 5 rows of transformed features after PCA:")
    print(X[:5])

    return X, y, scaler, pca

# Run the preprocessing function
X_processed, y_labels, scaler_model, pca_model = preprocess_data(full_dataset)
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y_labels, test_size=0.3, random_state=42)



Removing rows with NaN values...
Dataset shape after removing NaNs: (19358, 41)
First 5 rows of X (after NaN removal):
[[-3.17326879e-01  8.27515384e-01  7.60604845e-01  7.40965975e-01
   7.21417838e-01  9.10647001e-01  8.61356432e-01  8.35196209e-01
   8.15543437e-01  8.18339071e-01  7.58361024e-01  7.32744946e-01
   7.12313450e-01  8.06705401e-01  7.59407793e-01  1.60797637e-01
   2.14583946e-01  1.91990069e-01  1.82255482e-01  1.76242578e-01
  -2.84393560e-01 -1.14700224e-01 -5.35871230e-02 -8.57756000e-04
  -2.30281000e-04 -4.37930600e-03  3.93935000e-03 -2.54409068e-01
  -1.58545180e-01 -4.41293100e-02 -6.17937400e-03 -1.58812800e-03
  -4.25697300e-03 -2.38575000e-04 -2.66942911e-01 -1.06837230e-01
  -5.97025860e-02 -1.51620180e-02 -6.72895500e-03 -4.32901300e-03
   1.18994800e-03]
 [-5.03110538e-01  8.62969506e-01  8.02899468e-01  7.75813071e-01
   7.50999908e-01  9.27451720e-01  8.89261157e-01  8.66067459e-01
   8.48225700e-01  8.55546058e-01  8.02765823e-01  7.72049795e-01
   

## Step 4: Train the Classifier (Random Forest)

### Input
- `X_train`: Feature matrix for training (output from `train_test_split`)
- `y_train`: Label array corresponding to `X_train` (`0 = Cover`, `1 = Stego`)

### Output
- `clf`: Trained Random Forest Classifier model.

### Brief Explanation
This function uses the training portion of the preprocessed dataset to build a machine learning model that can detect steganography.

1. **Random Forest Classifier**:
   - An ensemble-based model that combines many decision trees to improve accuracy and avoid overfitting.
   - We use 100 trees (`n_estimators=100`) and limit each tree to a maximum depth of 10 (`max_depth=10`).

2. **Model Training**:
   - The classifier is trained using the `fit()` function on the training data.

3. **Evaluation on Training Set**:
   - **Training Accuracy**: Measures how well the model performs on the training data.
   - **Classification Report**: Provides detailed performance for each class (`Cover`, `Stego`) including:
     - Precision: How many predicted positives were actually correct.
     - Recall: How many actual positives were correctly predicted.
     - F1-score: Harmonic mean of precision and recall.


In [6]:

# Step 4: Function to train the classifier
from sklearn.metrics import classification_report

def train_classifier(X_train, y_train):
    """
    Trains a Random Forest Classifier using the provided training features and labels.

    Parameters:
    - X_train: Features for training
    - y_train: Labels for training

    Returns:
    - clf: Trained Random Forest Classifier
    - train_accuracy: Accuracy on the training set
    - train_report: Classification report (as string) for training performance
    """
    print("\nTraining Random Forest Classifier...")

    # Initialize classifier
    clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )

    # Train the model
    clf.fit(X_train, y_train)

    # Predict on training set to evaluate performance
    train_preds = clf.predict(X_train)
    train_accuracy = accuracy_score(y_train, train_preds)
    train_report = classification_report(y_train, train_preds, target_names=["Cover", "Stego"])

    # Display results
    print(f"\nTraining Accuracy: {train_accuracy:.4f}")
    print("\nClassification Report on Training Set:")
    print(train_report)

    return clf, train_accuracy, train_report

clf, accuracy, report = train_classifier(X_train, y_train)



Training Random Forest Classifier...

Training Accuracy: 0.7801

Classification Report on Training Set:
              precision    recall  f1-score   support

       Cover       0.90      0.63      0.74      6759
       Stego       0.72      0.93      0.81      6791

    accuracy                           0.78     13550
   macro avg       0.81      0.78      0.78     13550
weighted avg       0.81      0.78      0.78     13550



## Step 5: Extract CF Features from an Image

- **Input:**
  - `image_array`: A grayscale image in 2D format (512x512 pixels).
  - `scaler`: A trained StandardScaler model used to normalize the 41 features.
  - `pca`: A trained PCA model used to reduce the 41 features to 10.

- **Output:**
  - `pca_features`: A 10-value vector used in training/classification.
  - `raw_features`: A 41-value feature vector containing all extracted features for analysis.

- **Purpose:**
  This function extracts 41 statistical and correlation-based features from an image to detect steganography. The full feature vector is normalized and reduced using PCA to produce the 10 most important features for classification.

- **Used In:**
  Machine learning model training and prediction.


In [7]:
# Function to extract CF features from an image
import cv2
import numpy as np
from scipy import ndimage
from scipy.stats import pearsonr
import pywt
import requests
from io import BytesIO

# --- Helper Functions ---
def getPlaneBits(plane_id, binary_image):
    return [int(b[plane_id]) for b in binary_image]

def getBitPlanes(img):
    bin_image = [np.binary_repr(pixel, width=8) for row in img for pixel in row]
    bit_planes = [np.array(getPlaneBits(i, bin_image)).reshape(img.shape) for i in range(8)]
    return bit_planes

def autocor(matrix, k, l):
    Xk = matrix[0:matrix.shape[0] - k, 0:matrix.shape[1] - l]
    Xl = matrix[k:matrix.shape[0], l:matrix.shape[1]]
    return pearsonr(Xk.flatten(), Xl.flatten())

def getCHl(hist, l):
    return pearsonr(hist[0:256 - l], hist[l:256])

def getModifiedWavelet(coefficients, threshold):
    coefficients[np.abs(coefficients) < threshold] = 0
    return coefficients

def getE(img, threshold):
    LL, (LH, HL, HH) = pywt.dwt2(img, 'haar')
    LH = getModifiedWavelet(LH, threshold)
    HL = getModifiedWavelet(HL, threshold)
    HH = getModifiedWavelet(HH, threshold)
    img_denoised = pywt.idwt2((LL, (LH, HL, HH)), 'haar')
    return img - img_denoised

def getCE(img, threshold, k, l):
    residual = getE(img, threshold)
    return autocor(residual, k, l)

# --- Main CF Extraction Function ---
def extract_cf_features(image_array, scaler, pca):
    """
    Extracts 41 correlation features (CF) from a given grayscale image array.
    Then applies StandardScaler and PCA to reduce dimensionality for classification.

    Parameters:
    - image_array: 2D array of grayscale image (512x512)
    - scaler: Trained StandardScaler for normalization
    - pca: Trained PCA model for dimensionality reduction

    Returns:
    - pca_features: PCA-reduced feature vector (used for prediction)
    - raw_features: Original 41 extracted features (for analysis or display)
    """
    features = []
    bit_planes = getBitPlanes(image_array)

    # Bit plane correlation
    M1, M2 = bit_planes[0], bit_planes[1]
    features.append(pearsonr(M1.flatten(), M2.flatten())[0])

    # Autocorrelation on LSB
    autocor_pairs = [
        [1, 0], [2, 0], [3, 0], [4, 0],
        [0, 1], [0, 2], [0, 3], [0, 4],
        [1, 1], [2, 2], [3, 3], [4, 4],
        [1, 2], [2, 1]
    ]
    for k, l in autocor_pairs:
        features.append(autocor(M1, k, l)[0])

    # Histogram-based correlations
    img_hist, _ = np.histogram(image_array.flatten(), bins=256, density=True)
    He = img_hist[::2]  # Even indexed bins
    Ho = img_hist[1::2]  # Odd indexed bins
    features.append(pearsonr(He, Ho)[0])

    for i in range(1, 5):
        features.append(getCHl(img_hist, i)[0])

    # Wavelet residual correlations
    wavelet_triplets = [
        [1.5, 0, 1], [1.5, 1, 0], [1.5, 1, 1], [1.5, 0, 2], [1.5, 2, 0], [1.5, 1, 2], [1.5, 2, 1],
        [2.0, 0, 1], [2.0, 1, 0], [2.0, 1, 1], [2.0, 0, 2], [2.0, 2, 0], [2.0, 1, 2], [2.0, 2, 1],
        [2.5, 0, 1], [2.5, 1, 0], [2.5, 1, 1], [2.5, 0, 2], [2.5, 2, 0], [2.5, 1, 2], [2.5, 2, 1]
    ]
    for t, k, l in wavelet_triplets:
        features.append(getCE(image_array, t, k, l)[0])

    # Final transformations
    raw_features = np.array(features)
    scaled_features = scaler.transform(raw_features.reshape(1, -1))
    pca_features = pca.transform(scaled_features)

    print("\nExtracted 41 Raw Features (Before PCA):")
    print(raw_features)
    print("\nTransformed 10 Features (After PCA):")
    print(pca_features)

    return pca_features, raw_features


### Step 7: Prediction Function Using CF Features

#### Function: `run_prediction(scaler, pca, clf)`

- **Input:**
  - `scaler`: The trained StandardScaler model used to normalize the 41 extracted features.
  - `pca`: The trained PCA model used to reduce the 41 features to 10.
  - `clf`: The trained machine learning classifier (Random Forest in this case).

- **What it Does:**
  - Prompts the user to input a URL pointing to a grayscale `.pgm` image of size 512x512.
  - Downloads and reads the image from the URL.
  - Validates the image format and dimensions.
  - Extracts 41 correlation-based features (CF features) from the image.
  - Uses the pre-trained scaler and PCA to preprocess the features.
  - Feeds the processed features into the trained classifier.
  - Outputs a prediction result:
    - **"Steg Image (LSB Matching Detected)"** if steganography is detected.
    - **"Cover Image (No LSB Matching)"** if the image is clean.

- **Output:**
  - A string label describing the prediction result.
  - Also prints the result to the console.

In [10]:
# --- Main Prediction Function Using Extracted Features ---
def run_prediction(scaler, pca, clf):
    """
    Prompts the user for an image URL, downloads and decodes the image,
    extracts features, and predicts whether it contains LSB steganography.
    """
    image_url = input("\nEnter the URL of the image to test (must be 512x512 grayscale .pgm): ")
    print("\nDownloading and processing image...")

    try:
        response = requests.get(image_url)
        image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_GRAYSCALE)

        if image is None or image.shape != (512, 512):
            raise ValueError("The input image must be a 512x512 grayscale image.")

        print("Extracting CF features and making prediction...")
        pca_features, _ = extract_cf_features(image, scaler, pca)
        prediction = clf.predict(pca_features)

        result = "Steg Image (LSB Matching Detected)" if prediction[0] == 1 else "Cover Image (No LSB Matching)"
        print("\nPrediction Result:", result)
        return result

    except Exception as e:
        print("\nError processing the image:", e)
        return None

run_prediction(scaler_model, pca_model, clf)



Enter the URL of the image to test (must be 512x512 grayscale .pgm): https://raw.githubusercontent.com/Sourish1997/steganalysis/master/bossbase_sample/10.pgm

Extracting CF features and making prediction...

Extracted 41 Raw Features (Before PCA):
[ 2.58275200e-01  9.35111754e-01  9.03582478e-01  8.80394761e-01
  8.62877461e-01  9.10851835e-01  8.62668220e-01  8.33752815e-01
  8.09551728e-01  8.86327627e-01  8.34720541e-01  8.01004051e-01
  7.73503724e-01  8.49097146e-01  8.68801937e-01  9.93969823e-01
  9.92772179e-01  9.76649826e-01  9.53612761e-01  9.27852998e-01
 -1.56583962e-01 -2.11656983e-01 -7.08112983e-02 -6.66297908e-04
  1.30698673e-03 -1.43037889e-03 -8.90913395e-04 -1.63511556e-01
 -1.84495350e-01 -8.20628058e-02  8.17566030e-03  5.40958057e-03
 -7.31183583e-03 -9.71859993e-04 -1.70195274e-01 -1.72459213e-01
 -8.36635310e-02  9.55293184e-03  1.06065100e-02 -1.11553850e-02
 -2.32802423e-03]

Transformed 10 Features (After PCA):
[[ 0.64193957  2.3636459   0.58906108  0.8916

'Steg Image (LSB Matching Detected)'


Explanation of the Process:
1. Input : The user provided the path to the link to the image for testing.
2. Feature Extraction : The program extracted features from the image using the CF (Correlation Features) feature set described in the project report. These features capture spatial information from the image, particularly focusing on the least significant bit planes.
3. Preprocessing : The extracted features were preprocessed to ensure compatibility with the trained model. This includes:

  3.1 Normalization using StandardScaler.

  3.2 Dimensionality reduction using Principal Component Analysis (PCA).
4. Prediction : The preprocessed features were passed to the trained voting ensemble model, which consists of parameter-tuned versions of MLP Classifier and AdaBoost models.
5. Output : The model predicted that the image does not contain LSB matching steganography, classifying it as a Cover Image .

Key Points from the Output:
1. Prediction : The model classified the image as a Cover Image , meaning no signs of LSB matching steganography were detected.
2. Confidence : While the exact confidence score is not provided in the output, the model's accuracy and F-score (as reported in the project) suggest a reliable prediction. The final model achieved an accuracy of 75.52% and an F-score of 79.30% , which is significantly better than the benchmark Gaussian Naïve Bayes model.

 Possible Scenarios:
1. True Negative : If the image is indeed a clean image without any steganography, the prediction is correct.
2. False Negative : If the image contains LSB matching steganography but was misclassified as a cover image, this would indicate a limitation of the model. However, given the high F-score of the model, such cases are less likely but not impossible.

 Limitations to Consider:
1. Image Size : The feature extraction process is designed for 512x512 grayscale images. If the input image does not meet this requirement, it may have been cropped or resampled, potentially affecting the prediction.
2. Overly Uniform Images : If the image is overly dark or bright, some CF features may result in NaN values, making it unsuitable for analysis. However, since the program completed the prediction, this issue likely did not occur here.